In [ ]:
# compare umap/numba.njit/nndescent in speed up deepview

In [1]:
# import modules
import umap
import sys
import time
import numba
import numpy as np

from deepvisualinsight.MMS import MMS
# prepare training data
content_path = "E:\\DVI_exp_data\\resnet18_cifar10"
sys.path.append(content_path)
from Model.model import *
net = resnet18()

classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")
mms = MMS(content_path, net, 10, 200, 10, 512, 10, classes, cmap="tab10", resolution=100, neurons=256, verbose=1, temporal=False, split=-1, advance_border_gen=True, alpha=0.8, attack_device="cuda:0")

# shape (50000,3,32,32)
training_data = mms.training_data.cpu().numpy()
# shape (50000)
training_labels = mms.training_labels.cpu().numpy()
training_data = training_data.reshape(50000,-1)

Finish loading content!


In [5]:
# umap with euclidean distance, with numba.njit and nndescent
reducer = umap.UMAP(n_components=2, metric="euclidean")
t0 = time.time()
out = reducer.fit_transform(training_data)
t1 = time.time()
print("{:.2f} seconds.".format(t1-t0))


30.67 seconds.


In [10]:
@numba.njit()
def euclidean_numba(x1,x2):
    return np.linalg.norm(x1-x2)
# umap with euclidean distance, with numba.njit and nndescent
reducer = umap.UMAP(n_components=2, metric=euclidean_numba)
t0 = time.time()
out = reducer.fit_transform(training_data)
t1 = time.time()
print("{:.2f} seconds.".format(t1-t0))

D:\Downloads\anaconda\envs\DR2\lib\site-packages\umap\umap_.py:1623: UserWarning: custom distance metric does not return gradient; inverse_transform will be unavailable. To enable using inverse_transform method method, define a distance function that returns a tuple of (distance [float], gradient [np.array])
  "custom distance metric does not return gradient; inverse_transform will be unavailable. "


60.23 seconds.


In [ ]:
# @numba.njit()
def euclidean_numba(x1,x2):
    return np.linalg.norm(x1-x2)
# umap with euclidean distance, with numba.njit and nndescent
reducer = umap.UMAP(n_components=2, metric=euclidean_numba)
t0 = time.time()
out = reducer.fit_transform(training_data)
t1 = time.time()
print("{:.2f} seconds.".format(t1-t0))

In [26]:
@numba.njit()
def euclidean_numba(x1,x2):
    res = np.power(x1-x2,2)
    return np.sqrt(np.sum(res))

# umap with euclidean distance, with numba.njit and nndescent
reducer = umap.UMAP(n_components=2, metric=euclidean_numba)
t0 = time.time()
out = reducer.fit_transform(training_data)
t1 = time.time()
print("{:.2f} seconds.".format(t1-t0))

D:\Downloads\anaconda\envs\DR2\lib\site-packages\umap\umap_.py:1623: UserWarning: custom distance metric does not return gradient; inverse_transform will be unavailable. To enable using inverse_transform method method, define a distance function that returns a tuple of (distance [float], gradient [np.array])
  "custom distance metric does not return gradient; inverse_transform will be unavailable. "


62.59 seconds.


In [3]:
# @numba.njit()
def euclidean_server(x1,x2):
    return np.linalg.norm(x1-x2)
# umap with euclidean distance, with numba.njit and nndescent
reducer = umap.UMAP(n_components=2, metric=euclidean_server)
t0 = time.time()
out = reducer.fit_transform(training_data)
t1 = time.time()
print("{:.2f} seconds.".format(t1-t0))

D:\Downloads\anaconda\envs\DR2\lib\site-packages\umap\umap_.py:1623: UserWarning: custom distance metric does not return gradient; inverse_transform will be unavailable. To enable using inverse_transform method method, define a distance function that returns a tuple of (distance [float], gradient [np.array])
  "custom distance metric does not return gradient; inverse_transform will be unavailable. "


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at D:\Downloads\anaconda\envs\DR2\lib\site-packages\pynndescent\pynndescent_.py (361)[0m
[1m
File "D:\Downloads\anaconda\envs\DR2\lib\site-packages\pynndescent\pynndescent_.py", line 361:[0m
[1mdef nn_descent(
    <source elided>

[1m    if init_graph[0].shape[0] == 1:  # EMPTY_GRAPH
[0m    [1m^[0m[0m

This error may have been caused by the following argument(s):
- argument 4: [1mcannot determine Numba type of <class 'function'>[0m


In [4]:
import socket               # 导入 socket 模块
@numba.njit()
def euclidean_server(x1,x2):
    s = socket.socket()         # 创建 socket 对象
    host = socket.gethostname() # 获取本地主机名
    port = 12345                # 设置端口号

    s.connect((host, port))
    query = np.vstack((x1,x2))
    s.send(query.tostring())
    ans = s.recv(64)
    s.close()
    ans = np.fromstring(ans, dtype=np.float64)[0]

    return ans
a = euclidean_server(training_data[0],training_data[1])

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mUnknown attribute 'socket' of type Module(<module 'socket' from 'D:\\Downloads\\anaconda\\envs\\DR2\\lib\\socket.py'>)
[1m
File "<ipython-input-4-2a60c0cf3ac3>", line 5:[0m
[1mdef euclidean_server(x1,x2):
[1m    s = socket.socket()         # 创建 socket 对象
[0m    [1m^[0m[0m
[0m
[0m[1mDuring: typing of get attribute at <ipython-input-4-2a60c0cf3ac3> (5)[0m
[1m
File "<ipython-input-4-2a60c0cf3ac3>", line 5:[0m
[1mdef euclidean_server(x1,x2):
[1m    s = socket.socket()         # 创建 socket 对象
[0m    [1m^[0m[0m


In [ ]:
dummy = np.random.rand(50000,50000)
dummy = np.triu(dummy)
dummy = dummy+np.triu(dummy, -1).transpose()

In [ ]:
reducer = umap.UMAP(n_components=2, metric="precomputed")
t0 = time.time()
out = reducer.fit_transform(dummy)
t1 = time.time()
print("{:.2f} seconds.".format(t1-t0))